Here I'm going to basically try to predict the missing values in the "Market Category" column

In [1]:
import numpy as np
import pandas as pd
from numpy import mean, std
from main import MultiLabelEncoder

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df['Engine Fuel Type'].fillna(method='ffill', inplace=True)
df['Engine HP'].fillna(df['Engine HP'].median(), inplace = True)
df['Engine Cylinders'].fillna(df['Engine Cylinders'].mean(), inplace = True)
df['Number of Doors'].fillna(method='ffill', inplace = True)
df.drop('Model', axis=1, inplace=True)

In [4]:
#create new df containg rows with missing market category (test_df)
prediction_df = df.loc[df['Market Category'].isna()]
#drop rows with missing market category
df.drop(df.index[df['Market Category'].isna()], inplace=True)
#drop market category column
prediction_df.drop('Market Category', axis=1, inplace=True)

/tmp/ipykernel_88861/3728987043.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction_df.drop('Market Category', axis=1, inplace=True)


In [5]:
df_y = df.pop('Market Category')
encoder = MultiLabelEncoder(multilabel_column=df_y)

In [6]:
encoder.extract_unique_labels()
encoder.multilabel_oh_encode()
df_y_encoded = encoder.encodings

In [7]:
df['y_axis'] = df_y_encoded

In [8]:
df.to_csv('processed_data.csv', index=False)